In [25]:
import pandas as pd

# Sample data for orders
orders_data = [
    {'city': 'New York', 'customer_id': 1, 'sales': 100000},
    {'city': 'Los Angeles', 'customer_id': 2, 'sales': 9000},
    {'city': 'Chicago', 'customer_id': 3, 'sales': 8000},
    {'city': 'Houston', 'customer_id': 4, 'sales': 7000},
    {'city': 'Phoenix', 'customer_id': 5, 'sales': 6000},
    {'city': 'Philadelphia', 'customer_id': 1, 'sales': 5000},
    {'city': 'San Antonio', 'customer_id': 2, 'sales': 4000},
    {'city': 'San Diego', 'customer_id': 3, 'sales': 3000},
    {'city': 'Dallas', 'customer_id': 4, 'sales': 2000},
    {'city': 'San Jose', 'customer_id': 5, 'sales': 1000},
     {'city': 'New York', 'customer_id': 11, 'sales': 10000},
    {'city': 'Los Angeles', 'customer_id': 9, 'sales': 9000},
    {'city': 'Chicago', 'customer_id': 13, 'sales': 8000},
    {'city': 'Houston', 'customer_id': 14, 'sales': 7000},
    {'city': 'Phoenix', 'customer_id': 7, 'sales': 6000},
    {'city': 'Philadelphia', 'customer_id': 8, 'sales': 5000},
    {'city': 'San Antonio', 'customer_id': 3, 'sales': 4000},
    {'city': 'San Diego', 'customer_id': 16, 'sales': 3000},
    {'city': 'Dallas', 'customer_id': 14, 'sales': 2000},
    {'city': 'San Jose', 'customer_id': 55, 'sales': 1000},
]

# Create a pandas dataframe for orders
orders_df = pd.DataFrame(orders_data)

In [26]:
# Sample data for customers
customers_data = [
    {'customer_id': 1, 'city': 'New York'},
    {'customer_id': 2, 'city': 'Los Angeles'},
    {'customer_id': 3, 'city': 'Chicago'},
    {'customer_id': 4, 'city': 'Houston'},
    {'customer_id': 5, 'city': 'Phoenix'},
    {'customer_id': 6, 'city': 'Philadelphia'},
    {'customer_id': 7, 'city': 'San Antonio'},
    {'customer_id': 8, 'city': 'San Diego'},
    {'customer_id': 9, 'city': 'Dallas'},
    {'customer_id': 10, 'city': 'San Jose'},
]

# Create a pandas dataframe for customers
customers_df = pd.DataFrame(customers_data)

In [27]:
import sqlite3

# Connect to in-memory database
conn = sqlite3.connect(':memory:')


# Create a table in the database
orders_df.to_sql("orders", conn, if_exists="replace")
customers_df.to_sql("customers", conn, if_exists="replace")

Following query uses two subqueries, top_cities and top_customers which are defined using the WITH clause. 
The first subquery top_cities retrieves the top 10 cities with the highest total sales, based on the data in the "orders" table. 
The second subquery top_customers retrieves the top 5 customers with the highest total sales, also based on the data in the "orders" table.
The outer query then joins the two subqueries top_cities and top_customers on the city column and filters the results based on the total sales 
of the top cities and customers, and also uses a subquery inside the join clause to match the city of the customer with the cities in the top_cities subquery.


Use of named sub or use of the WITH clause can make the query more readable and maintainable.



In [28]:
query = """
  
WITH top_cities AS (
  SELECT city, SUM(sales) AS total_sales
  FROM orders
  GROUP BY city
  ORDER BY total_sales DESC
  LIMIT 10
),
top_customers AS (
  SELECT customer_id, SUM(sales) AS total_sales
  FROM orders
  GROUP BY customer_id
  ORDER BY total_sales DESC
  LIMIT 5
)
SELECT top_cities.city, top_customers.customer_id, top_customers.total_sales
FROM top_cities
JOIN top_customers
ON top_cities.city = (SELECT city FROM customers WHERE customers.customer_id = top_customers.customer_id)
WHERE top_cities.total_sales > 100000
ORDER BY top_cities.city, top_customers.total_sales DESC;

    
"""


In [29]:
# use pandas read_sql to execute the query and return a dataframe
result_df = pd.read_sql(query, con=conn)

In [30]:
result_df

,city,customer_id,total_sales
0,New York,1,105000
